In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import time
from datetime import date

NEWSPAPER = 'Fox'
SECTIONS = ['entertainment', 'politics', 'us', 'world']
NEWSPAPER_URL = "https://www.foxnews.com"

In [2]:
db = pd.read_csv('db_current.csv')

In [3]:
article_body_collected = list(db['link_to_original'].values)
len(article_body_collected)

2212

In [4]:
today = date.today()
today = str(today).replace('-', '_')
print("Today's date:", today)

Today's date: 2023_03_28


In [5]:
scraped_today = ['WSJ', 'NYT']
for scraped_file in scraped_today:
    scraped_file_name = scraped_file + '_' + today + '.csv'
    temp_df = pd.read_csv('scrape_results\\' + today + '\\' + scraped_file_name)
    scraped_today_links = list(temp_df['link_to_original'].values)
    article_body_collected += scraped_today_links

In [6]:
len(article_body_collected)

2379

In [7]:
article_body_collected_set = set(article_body_collected)
len(article_body_collected_set)

2286

In [8]:
article_body_collected = list(article_body_collected_set)
len(article_body_collected)

2286

In [9]:
def fox_extract_article(article_link):
    web = requests.get(article_link)
    article_content = soup(web.content,'html.parser')
    
    article_content_processed = article_content.find("script", {"type": "application/ld+json"})
    article_title = json.loads(article_content_processed.text)['headline']
    article_dt = json.loads(article_content_processed.text)['datePublished'].split('T')[0]
    article_time = int(pd.to_datetime(json.loads(article_content_processed.text)['datePublished']).timestamp()) * 1000
    
    article_body = ''
    paragraphs = article_content.find("div", {"class": 'article-body'}).find_all('p')
    for p in paragraphs:
        if 'a href' and '<strong>' in str(p):
            pass
        elif '@fox.com' in str(p):
            pass
        elif ('Fox News\'' in str(p) and 'contributed to this report.' in str(p)):
            pass
        elif 'span' in str(p):
            pass
        else:
            #print(p.text)
            article_body += p.text + ' '

    return(article_title, article_body, article_dt, article_time)

In [10]:
%%time
list_article_title = []
list_article_link = []
list_article_body = []
list_article_dt = []
list_article_time = []
list_article_section = []

for section in SECTIONS:
    newspaper_section_url = NEWSPAPER_URL + "/" + section
    headers = {'User-Agent': 'Mozilla/5.0'}
    web = requests.get(newspaper_section_url, headers=headers)
    section_content = soup(web.content,'html.parser')
    
    articles = section_content.find_all("article", {"class": 'article'})
    for article in articles:
        if 'foxnews.com' in str(article.find('a')).split('<a href=\"')[1].split('\"><')[0]:
            article_link_ending = article_link_ending.split('foxnews.com')[0]
        else:
            article_link_ending = str(article.find('a')).split('<a href=\"')[1].split('\"><')[0]
        article_link = NEWSPAPER_URL + article_link_ending
        #article_link = article_link.replace('//', '/')
        if article_link in article_body_collected:
            pass
        else:
            if article_link in list_article_link:
                index_of_interest = list_article_link.index(article_link)
                if isinstance(list_article_section[index_of_interest], list):
                    if section in list_article_section[index_of_interest]:
                        pass
                elif section == list_article_section[index_of_interest]:
                        pass
                else:
                    if isinstance(list_article_section[index_of_interest], list):
                        list_article_section[index_of_interest].append(section)
                    else:
                        new_list = [list_article_section[index_of_interest]]
                        new_list.append(section)
                        list_article_section[index_of_interest] = new_list
            else:
                if ('/video/' not in article_link and
                    '/opinion/' not in article_link and
                    '/media/' not in article_link and
                    '/entertainment/' not in article_link and
                    '/sports/' not in article_link and
                    '/lifestyle/' not in article_link):

                    # print(article_link)
                    article_title, article_body, article_dt, article_time = fox_extract_article(article_link)

                    list_article_title.append(article_title)
                    list_article_link.append(article_link)
                    list_article_body.append(article_body)
                    list_article_dt.append(article_dt)
                    list_article_time.append(article_time)
                    list_article_section.append(section)

Wall time: 8.98 s


In [11]:
article_link_ending

'/world/britain-overhaul-clinical-trial-regulation-fast-track-approvals'

In [12]:
article_link_ending.split('foxnews.com')

['/world/britain-overhaul-clinical-trial-regulation-fast-track-approvals']

In [13]:
articles_dict = {'title':list_article_title,
    'link_to_original':list_article_link,
    'article_body':list_article_body,
    'article_dt':list_article_dt,
    'article_time':list_article_time,
    'section': list_article_section,
   }
    
df = pd.DataFrame(articles_dict)

In [18]:
filename = NEWSPAPER + '_' + today + '.csv'

In [19]:
df.to_csv('\\scrape_results\\' + today + '\\' + filename, index = False)